<a href="https://colab.research.google.com/github/afujii/class/blob/main/%E5%92%8C%E6%AD%8C%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 和歌の分析


以下のファイルを分析対象とする。このファイルは、縦書き一太郎から横書きUTF8コードテキストを生成し、成形したものである。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
daiuta = {}
with open('/content/drive/MyDrive/Colab Notebooks/daiuta.txt') as f:
  for line in lines:
    l = line.split(",")
    daiuta[l[0]] = l[1]
data = list(daiuta.items())
print(data[:3])

[('祝', '神代よりをしへのままのあととめて榮ひさしきしきしまの道\n'), ('題不知', 'みな人は上に目がつく横にゆく葦間のかにのあはれなる世や\n'), ('後陽成院五日百首', 'ちかひなほ世世にかはらで八嶋もる國つみ神は頼もしきかな\n')]


次に分析用の形態素解析環境を整える。Janome、Spacy、MeCabなどがある。以下では、MeCapの一般的な辞書をつかう例を示す。

In [ ]:
!pip install mecab-python3
!pip install unidic

In [ ]:
!python -m unidic download

MeCabを利用した「形容詞」「名詞」「動詞」による分かち書きの例を以下に示す。頻出単語を示す。古語辞書を利用する場合と語彙が異なる。その比較をColabの上で示すのは煩雑なので省略する。
分かち書きした後、語彙の頻出50位までを示す。

In [ ]:
import MeCab
tagger = MeCab.Tagger()

# 歌ごとにparse
words = {}
for uta in data:
  res = tagger.parse(uta[1])
  tags = res.split('\n')
  for t in tags:
    elements = t.split(',')[0].split('\t')
    if len(elements)>1 and \
       (elements[1]=='形容詞' or \
        elements[1]=='名詞' or \
        elements[1]=='動詞'):
      if elements[0] not in words.keys():
        words[elements[0]]=0
      words[elements[0]] += 1
stat = sorted(words.items(),key= lambda x : x[1],reverse = True)
for i in stat[0:30]:
  print(i[0]+"\t\t"+str(i[1]))

神		29
春		29
し		28
き		26
世		24
月		20
山		19
道		18
國		15
なき		15
ひ		15
花		15
神代		14
人		13
葉		13
千代		13
秋		12
心		11
年		11
いく		11
こと		10
見		10
色		10
世世		9
松		9
よろづ		9
ひかり		9
ぎ		9
こころ		8
露		8


以降、これまで利用した daiuta.txt ファイルを用いる。機械学習として、2つの分析を試みる。第１は、　word2vec による学習。第2はLSTMによる学習である。別なファイルで紹介する。
